<a href="https://colab.research.google.com/github/lbal-biomat/quality/blob/main/log.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

El paper en el que me guío es:

Nicholls, S. M., Quick, J. C., Tang, S., & Loman, N. J. (2019). Ultra-deep, long-read nanopore sequencing of mock microbial community standards. Gigascience, 8(5), giz043. https://academic.oup.com/gigascience/article/8/5/giz043/5486468

La idea es seguir los pasos del paper y ver que todo es reproducible, luego ensamblar con flye los últimos datos que publicaron (R10.3) y hacer el polish con los protocolos más recomendados, todos usando la calidad en algún punto

- Flye + racon x1
- Flye + racon x1 + medaka
- Flye + racon x1 + homopolish
- Flye + racon x1 + medaka + homopolish
- Flye + racon x2
- Flye + racon x2 + medaka
- Flye + racon x2 + homopolish
- Flye + racon x2 + medaka + homopolish
- Flye + racon x4
- Flye + racon x4 + homopolish
- Flye + MarginPolish + helen
- Flye + MarginPolish + homopolish
- Flye + MarginPolish + helen + homopolish

racon y marginpolish/helen usan información de calidad por base.

 <br/>

**Instalación de programas**

Instalé las versiones correspondientes al paper con conda:

```
conda create --prefix ./env_paper python=3.6 pilon=1.23 racon=1.3.2 minimap2=2.14 kraken2 checkm-genome quast=5.0.2
```
`wtdbg2` no estaba disponible en la versión 2.2, por lo que bajé los ejecutables de https://github.com/ruanjue/wtdbg2/releases/download/v2.2/wtdbg-2.2_x64_linux.tar.gz

`medaka` tampoco estaba disponible en la versión 0.5.0, por lo que lo instalé desde la fuente en un environment aparte.
Para eso descargue los archivos de https://pypi.org/project/medaka/0.5.0/#files. La versión correcta de medaka es importante porque la nueva no tiene el modelo correspondiente al basecaller que usaron en el paper.

Antes de hacer make install a medaka hay que cambiar algunas dependencias del archivo `requirements.txt`:
- numpy==1.19.2 en lugar de numpy, y pasar esa linea arriba de la de tensorflow
- sacar la version a tensorflow (la que pide, 1.x.x, ya no está disponible como libreria de python)

 <br/>

Instalé las últimas versiones de los programas en otro environment:

```
conda create --prefix ./env_nuevos flye=2.8.2 racon=1.4.13 medaka=1.2.1 minimap2=2.17 homopolish=0.0.1
```
homopolish tiene problemas de compatibilidad con todo así que lo puse en un env aparte:

```
conda create --prefix ./hmp homopolish 
```

 <br/>

**Datos y descarga**

Zymo Community Standards 2 (Even) Batch ZRC190633:
10 species (5 Gram-positive, 3 Gram-negative, 2 yeast): the bacteria are present at 12% and yeast at 2% (by genomic DNA)

Zymo Community Standards 2 (Log/Staggered) Batch ZRC190842
10 species (5 Gram-positive, 3 Gram-negative, 2 yeast) ranging from 10^2 - 10^8 genomic DNA abundance (total input 5 x 10^8 cells)

| Community | # of Spots    | # of Bases | Size |
| ----------|:-------------:| ----------:|-----:|
| Even      | 3,491,390     | 14.4G      | 12Gb | 
| Log       | 	3,667,480   |   16.5G    | 13.7Gb |


		  	      


  	      
Esos son los datos del poro 9.41 secuenciados con la plataforma GridION (para el pipeline del paper). Descargué los fastQ de https://github.com/LomanLab/mockcommunity.

En el paper también usan los datos secuenciados con PromethION. Estos son muy pesados porque tienen mucha profundidad. En principio no los voy a usar.

También están los datos de señal, en principio no los descargué, para seguir los pasos del paper estoy trabajando con los datos tal cual los de ellos.

Descargué los datos de poro R10.3 de https://nanopore.s3.climb.ac.uk/mock/Zymo-GridION-EVEN-3Peaks-R103-merged.fq.gz. Son pocos reads (~1.16M), cuando tenga acceso a torito voy a ver de hacer una prueba con los datos que tienen publicados de poro R10 que son muchos mas reads (6.74M), disponibles en https://nanopore.s3.climb.ac.uk/mock/Zymo-GridION-Even-3Peaks-Native-R10_hac_meth.fq.gz.

Están publicados los assemblies de spades con reads de illumina: https://github.com/LomanLab/mockcommunity/blob/master/analysis/illumina/spades/Zymo-Isolates-SPAdes-Illumina.fasta.gz (aún no los bajé, no estoy segura de necesitarlos).

Descargué los reads de la secuenciación illumina de metagenome de la comunidad even para el polishing con pilon del SRA: ERR2984773.

Me falta descargar la base de datos de microbios de kraken2:

```
wget -c https://refdb.s3.climb.ac.uk/kraken2-microbial/hash.k2d
wget https://refdb.s3.climb.ac.uk/kraken2-microbial/opts.k2d
wget https://refdb.s3.climb.ac.uk/kraken2-microbial/taxo.k2d
```

**Notas**

Racon no anda bien con los reads comprimidos, es mejor dárselos sin comprimir. 

minimap2 y racon no se pueden usar con nohup porque la salida estándar va a parar al archivo final. Se puede poner el comando dentro de un script y llamar al script con nohup.

El primer intento de correr racon me dio el siguiente error:
```
[racon::Polisher::initialize] error: empty overlap set!
```
Hay que borrar el newline al final del archivo .paf de minimap2, y voila!

La versión 0.5.0 de medaka tiene problemas de incompatibilidad con la nueva versión de tensorflow. 
Tuve que reemplazar `tf` por `tf.compat.v1` en varios lugares de varios scripts (todos los que iban dando error), esos archivos fueron:
- /media3/lucia/medaka-0.5.0/medaka-0.5.0/venv/lib/python3.8/site-packages/keras/backend/tensorflow_backend.py
- /media3/lucia/medaka-0.5.0/medaka-0.5.0/venv/lib/python3.8/site-packages/medaka-0.5.0-py3.8-linux-x86_64.egg/medaka/inference.py

En el archivo /media3/lucia/medaka-0.5.0/medaka-0.5.0/venv/lib/python3.8/site-packages/medaka-0.5.0-py3.8-linux-x86_64.egg/medaka/common.py tuve que cambiar, en la linea 443, `raise StopIteration` por `return`. Esto es porque en la última versión de python cambió cómo se usan los generators.

Con esos cambios pude correr el programa.

Estoy teniendo problemas para correr homopolish